In [1]:
import sys
if not '..' in sys.path:
    sys.path.append('..')

In [2]:
from analysis import collection, analysis

from pathlib import Path

import numpy as np
import scipy
from analysis import analysis
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection
from matplotlib.colors import to_rgba
from IPython.display import HTML

from matplotlib.gridspec import GridSpec

import matplotlib as mpl
mpl.rcParams.update({'axes.labelsize': 16})

%matplotlib inline

image_path = Path("/Users/spencer/phd/phd/images/")

In [3]:
collection_name = "emg_olympics"
task_name = "center_hold"
c = collection.Collection(collection_name)
subjects = c.alphabetically_sorted_subjects()

## inspect natural data

In [ ]:
### load one trial of natural movement data

filenames = get_movement_filenames("emg_olympics", test_subject.name, "natural_movement", "session_0")
emg = load_movement_emg(filenames[6])
filtered = filter_emg(emg, test_subject.variance)
indices, low_point, mean_norm_threshold = find_nat_movement_active_indices(filtered)
# 1000, 64
X = filtered[indices]
# 1000, 2
projected_movement = X @ test_subject.decoder[-2:,:].T


### trial data

trial = test_subject.tasks[task_name].sorted_sessions()[0].sorted_trials()[4]
X = trial.get_filtered_emg()[trial.active_indices]
T = X @ test_subject.decoder[-2:,:].T

print(projected_movement.shape, T.shape)


## make natural movement data brick over subjects

In [ ]:
# subject_movement_stacks = np.load("subject_natural_movement_stacks.npy")
# subject_movement_stacks.shape

def find_nat_movement_active_indices(sig, std_multiple=1.0, min_indices=10):
    assert sig.shape[1] == 64
    sig_norms = np.linalg.norm(sig,axis=1)

    # find location of min trajectory norm (closest to zero)
    low_point = np.argmin(sig_norms)

    # stddev of the spatial norms
    sig_norm_std = np.std(sig_norms)

    # compute the threshold, mean of a short range around the low point
    mean_norm_threshold = np.mean(
        sig_norms[np.max([0, low_point - 5]) : low_point + 5], axis=0
    )

    # mask -- one std above the lowest norm point
    mask = sig_norms > mean_norm_threshold + sig_norm_std * std_multiple

    # return the indices
    if np.sum(mask) < min_indices:
        indices = np.arange(sig.shape[0])[-min_indices:]
    else:
        indices = np.arange(sig.shape[0])[mask]
    return indices, low_point, mean_norm_threshold


def load_jagged_movement_stack(subject):

    # more than enough room for the active indices
    movement_emg = np.empty(shape=(2,1500,64))
    movement_emg[:] = np.nan

    for session_idx, session_name in enumerate(["session_0", "session_1"]):
        filenames = get_movement_filenames("emg_olympics", subject.name, "natural_movement", session_name)
        for movement_idx, filename in enumerate(filenames):
            emg = load_movement_emg(filename)
            filtered = emg.filter_emg(emg, subject.variance)
            indices, low_point, mean_norm_threshold = find_nat_movement_active_indices(filtered)
            movement_emg[session_idx,:indices.shape[0],:] = filtered[indices]

    return movement_emg

natural_movement_stack = load_jagged_movement_stack(test_subject)
print(natural_movement_stack.shape)



## save it to disk!